# Parameter Transformation Example

This notebook demonstrates how to use the `LocalTransformerClient` to transform input parameters for LLM requests based on model-specific configurations.

In [1]:
import os
from pprint import pprint as pp

# Optional environment variables
# os.environ["LANGGATE_CONFIG"] = "path/to/langgate_config.yaml"
# os.environ["LOG_LEVEL"] = "debug"

## Initialize the Transformer Client

In [2]:
from langgate.transform import LocalTransformerClient

# Initialize the transformer client
transformer = LocalTransformerClient()

2025-07-09 23:43:51 [debug    ] resolved_config_path           exists=True path=/Users/someuser/langgate/examples/langgate_config.yaml source=default
2025-07-09 23:43:51 [debug    ] resolved_env_file_path         exists=False path=/Users/someuser/langgate/examples/.env source=default
2025-07-09 23:43:51 [info     ] loaded_config                  config_path=/Users/someuser/langgate/examples/langgate_config.yaml
2025-07-09 23:43:51 [debug    ] initialized_local_transformer_client


## Define Input Parameters

These are the parameters that would typically be provided by your application code.

In [3]:
# Sample user parameters
input_params = {
    "temperature": 0.7,
    "max_tokens": 1000,
    "stream": True,
}

## Transform Parameters for a Specific Model

The transformer will apply transformations based on the model configuration in `langgate_config.yaml`.

In [4]:
# Transform parameters for an Anthropic model
model_id = "anthropic/claude-sonnet-4"

transformed = await transformer.get_params(model_id, input_params)

print(f"Original parameters for {model_id}:")
pp(input_params)

print("\nTransformed parameters:")
pp(transformed)

Original parameters for anthropic/claude-sonnet-4:
{'max_tokens': 1000, 'stream': True, 'temperature': 0.7}

Transformed parameters:
('anthropic',
 {'api_key': SecretStr('**********'),
  'base_url': 'https://api.anthropic.com',
  'max_tokens': 1000,
  'model': 'claude-sonnet-4-0',
  'stream': True,
  'temperature': 0.7})


## Working with Model-Specific Configurations

Different models may have different parameter transformations defined in `langgate_config.yaml`.
You can define "virtual models" with IDs referring to a model + a configuration.

In [5]:
# Using a model with a specific configuration.
# Anthropic has no actual model with this ID. We define it ourselves,
# to reference claude-sonnet-4 with a specific reasoning configuration.
model_id = "anthropic/claude-sonnet-4-reasoning"

transformed = await transformer.get_params(model_id, input_params)

print(f"Original parameters for {model_id}:")
pp(input_params)

print("\nTransformed parameters:")
pp(transformed)

Original parameters for anthropic/claude-sonnet-4-reasoning:
{'max_tokens': 1000, 'stream': True, 'temperature': 0.7}

Transformed parameters:
('anthropic',
 {'api_key': SecretStr('**********'),
  'base_url': 'https://api.anthropic.com',
  'max_tokens': 1000,
  'model': 'claude-sonnet-4-0',
  'stream': True,
  'thinking': {'budget_tokens': 1024, 'type': 'enabled'}})


## Transform Parameters for Image Models

The transformer also works with image generation models.

In [6]:
# Image generation parameters
image_params = {
    "prompt": "A serene landscape with mountains",
    "size": "1024x1024",
    "quality": "standard",
}

# Transform parameters for an image model
image_model_id = "openai/dall-e-3"

transformed = await transformer.get_params(image_model_id, image_params)

print(f"Original parameters for {image_model_id}:")
pp(image_params)

print("\nTransformed parameters:")
pp(transformed)

Original parameters for openai/dall-e-3:
{'prompt': 'A serene landscape with mountains',
 'quality': 'standard',
 'size': '1024x1024'}

Transformed parameters:
('openai',
 {'api_key': SecretStr('**********'),
  'base_url': 'https://api.openai.com/v1',
  'model': 'dall-e-3',
  'prompt': 'A serene landscape with mountains',
  'quality': 'standard',
  'size': '1024x1024'})


In [7]:
# Transform parameters for a different image model provider
replicate_model_id = "black-forest-labs/flux-dev"

# Different providers may have different parameter formats
replicate_params = {
    "prompt": "A futuristic cityscape at night",
    "width": 1024,
    "height": 1024,
    "num_inference_steps": 30,
}

transformed = await transformer.get_params(replicate_model_id, replicate_params)

print(f"Original parameters for {replicate_model_id}:")
pp(replicate_params)

print("\nTransformed parameters:")
pp(transformed)

2025-07-09 23:44:21 [warning  ] api_key_env_var_not_found      service=replicate variable=REPLICATE_API_KEY
Original parameters for black-forest-labs/flux-dev:
{'height': 1024,
 'num_inference_steps': 30,
 'prompt': 'A futuristic cityscape at night',
 'width': 1024}

Transformed parameters:
('replicate',
 {'disable_safety_checker': True,
  'height': 1024,
  'model': 'black-forest-labs/flux-dev',
  'num_inference_steps': 30,
  'prompt': 'A futuristic cityscape at night',
  'width': 1024})
